In [ ]:
import numpy 
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torch.optim import SGD, Adam, RMSprop
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
print(f"torch.cuda.is_available(): {torch.cuda.is_available()}")

torch.manual_seed(0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# [Task 1]: Import and initialize ResNet50
resnet_corolectal_cancer_model = resnet50(weights=None)
'''
Note that by default the ResNet implementations from PyTorch's source code follow the ImaegNet fully connected layer specification of 2048x1000.
    Therefore, for our project, we need to change it to use 3 classes according to the first dataset (MUS, NORM, STR).
    See the source code for the base class here (https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py).
    Stack Overflow post here (https://stackoverflow.com/questions/68980724/the-number-of-classes-in-pytorch-pretrained-model).

    Additionally, we will need to remove the last layer to be able to pass the features to a different learning algorithm in task 2.
    Here is a Stack Overflow post about it (https://stackoverflow.com/questions/52548174/how-to-remove-the-last-fc-layer-from-a-resnet-model-in-pytorch).

'''
resnet_corolectal_cancer_model.fc = torch.nn.Linear(2048, 3)
resnet_corolectal_cancer_model.to(device=device)
resnet_corolectal_cancer_model.train()

# [Task 1]: Import and initialize the optimizer algorithms
'''
Hyperparameters for vanilla SGD recommended settings: learning rate = 0.001
                                                      L2 regularizer penalty = 1e-6
                                                      Batch Size = 100
'''
learning_rate = 0.001
l2_regularizer = 1e-6
batch_size = 100

sgd_no_momentum = SGD(resnet_corolectal_cancer_model.parameters(), lr=learning_rate, weight_decay=l2_regularizer, differentiable=True)

'''
Hyperparameters for SGD + momentum recommended settings: learning rate = 0.001
                                                         L2 regularizer penalty = 1e-6
                                                         Batch Size = 100
                                                         Momentum = 0.9
                                                         Nesterov = true
'''
sgd_momentum = 0.9
sgd_momentum_use_nesterov = True

sgd_momentum = SGD(resnet_corolectal_cancer_model.parameters(),
                   lr=learning_rate,
                   weight_decay=l2_regularizer,
                   momentum=sgd_momentum,
                   nesterov=sgd_momentum_use_nesterov,
                   differentiable=True)

'''
Hyperparameters for Adam recommended settings: learning rate = 0.001
                                               L2 regularizer penalty = 1e-6
                                               Batch Size = 100
                                               Beta 1 (Momentum) = 0.9
                                               Beta 2 (RMS decay rate) = 0.999
                                               Division stablizer = 1e-8
'''
adam_betas = (0.9, 0.999)
epsilon_division_stablizer = 1e-8

adam = Adam(resnet_corolectal_cancer_model.parameters(),
            lr=learning_rate,
            weight_decay=l2_regularizer,
            betas=adam_betas,
            eps=epsilon_division_stablizer,
            differentiable=True)

'''
Hyperparameters for RSMProp recommended settings: learning rate = 0.001
                                                  L2 regularizer penalty = 1e-6
                                                  Batch Size = 100
                                                  Decay rate = 0.999
                                                  Division stablizer = 1e-8
'''
rmsprop_decay_rate = adam_betas[1]

rmsprop = RMSprop(resnet_corolectal_cancer_model.parameters(),
                  lr=learning_rate,
                  weight_decay=l2_regularizer,
                  alpha=rmsprop_decay_rate,
                  eps=epsilon_division_stablizer,
                  differentiable=True)

# [Task 1]: Import and initialize the t-SNE visualization algorithm
tsne = TSNE(random_state=42)

In [ ]:
'''Optimizers'''

# optimizer = rmspro
# optimizer = SGD(resnet_corolectal_cancer_model.parameters(), lr=learning_rate, weight_decay=l2_regularizer)
# optimizer = Adam(resnet_corolectal_cancer_model.parameters(), lr=learning_rate, weight_decay=l2_regularizer)
optimizer = RMSprop(resnet_corolectal_cancer_model.parameters(), lr=learning_rate, weight_decay=l2_regularizer)

# Preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.ToTensor(),  # TensorFlow
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet Normalization
])

full_dataset = datasets.ImageFolder(root='Colorectal Cancer', transform=transform)

# Split 80:20, train/test
train_size = int(0.8 * len(full_dataset))  
test_size = len(full_dataset) - train_size  
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Dataloaders for each type
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


num_epochs = 1 

for epoch in range(num_epochs):  # Iterate over epochs
    resnet_corolectal_cancer_model.train()  # Set model to training mode
    epoch_loss = 0.0  # Track loss for the epoch

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device) 

        optimizer.zero_grad()  # Zero the gradients
        
        outputs = resnet_corolectal_cancer_model(inputs)  # Forward pass
        
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Calculate loss
        
        loss.backward()  # Backward pass to compute gradients
        optimizer.step()  # Update model parameters using gradients

        epoch_loss += loss.item()  

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_dataloader):.4f}")

# Eval 
resnet_corolectal_cancer_model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient tracking
    test_loss = 0.0 
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU if available
        
        outputs = resnet_corolectal_cancer_model(inputs)  # Forward pass
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Test loss
        
        test_loss += loss.item()  # Cumulative test loss

    average_test_loss = test_loss / len(test_dataloader)
    print(f"Test Loss: {average_test_loss:.4f}")

